In [4]:
import os, time, math
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

Ames_geo_original = pd.read_csv('Ames Real Estate Data.csv', index_col=0)
Ames_original = pd.read_csv('Ames_HousePrice.csv', index_col=0)

Ames_geo = Ames_geo_original.copy()
Ames = Ames_original.copy()

/var/folders/jp/xgt314z12mgc2qh1wbn4mbx40000gn/T/ipykernel_14996/1767887551.py:7: DtypeWarning: Columns (36,40,81,87) have mixed types. Specify dtype option on import or set low_memory=False.
  Ames_geo_original = pd.read_csv('Ames Real Estate Data.csv', index_col=0)


In [5]:
# Create a full address string for geocoding
Ames_geo["FullAddress"] = (
    Ames_geo["PA-Nmbr"].fillna("").astype(str).str.strip() + " " +
    Ames_geo["PA-Strt"].fillna("").str.strip() + " " +
    Ames_geo["PA-StSfx"].fillna("").str.strip() + " " +
    Ames_geo["PA-PostD"].fillna("").str.strip() + ", Ames, IA"
).str.replace("  ", " ").str.strip()

In [6]:
import pandas as pd

# 1) Load the two datasets
# Ames is your modeling dataset
# Ames_geo is the assessor dataset
# Assuming they are already loaded into Ames and Ames_geo

# 2) Ensure keys are same dtype (PID ↔ MapRefNo)
Ames["PID"] = Ames["PID"].astype(str)
Ames_geo = Ames_geo.reset_index()
Ames_geo["MapRefNo"] = Ames_geo["MapRefNo"].astype(str)

# 3) Filter assessor data to only rows in Ames
Ames_geo_filtered = Ames_geo[Ames_geo["MapRefNo"].isin(Ames["PID"])].copy()

print("Original assessor rows:", len(Ames_geo))
print("Filtered to Ames rows:", len(Ames_geo_filtered))

Original assessor rows: 22213
Filtered to Ames rows: 2602


In [12]:
# 0) Make copies so we don't mutate originals
ames = Ames.copy()
geo  = Ames_geo.copy()

# 1) Normalize join keys as *columns* (not index) → strings of ints with no decimals
ames["PID_key"] = pd.to_numeric(ames["PID"], errors="coerce").astype("Int64").astype(str)
geo["MapRefNo_key"] = pd.to_numeric(geo["MapRefNo"], errors="coerce").astype("Int64").astype(str)

print("Ames PID raw (col):", ames["PID"].head(5).tolist())
print("Geo MapRefNo raw (col):", geo["MapRefNo"].head(5).tolist())

# 2) Quick overlap sanity check
pid_set = set(ames["PID_key"].dropna())
geo_set = set(geo["MapRefNo_key"].dropna())
overlap = pid_set & geo_set
print("Overlap count:", len(overlap))
print("Overlap sample:", list(overlap)[:10])

# 3) Filter geo to only PIDs present in Ames
geo_filtered = geo[geo["MapRefNo_key"].isin(pid_set)].copy()
print("Filtered geo rows:", len(geo_filtered))

# 4) (Optional) build a clean full address string on the filtered geo for geocoding
import numpy as np
import pandas as pd

def _as_text(x):
    """Coerce any value to a clean string; handle NaN and floats like 4507.0 -> '4507'."""
    if pd.isna(x):
        return ""
    # numeric house numbers often come as floats with .0
    if isinstance(x, (int, np.integer)):
        return str(x)
    if isinstance(x, float):
        return str(int(x)) if x.is_integer() else str(x)
    return str(x)

def build_full_address(row):
    # Pull parts and sanitize
    num    = _as_text(row.get("PA-Nmbr"))
    pre    = _as_text(row.get("PA-PreD"))
    street = _as_text(row.get("PA-Strt"))
    sfx    = _as_text(row.get("PA-StSfx"))
    post   = _as_text(row.get("PA-PostD"))
    # Assemble, dropping empties / 'nan'
    parts = [p.strip() for p in [num, pre, street, sfx, post] if p and p.strip().lower() != "nan"]

    if not parts:
        # fall back to Prop_Addr if structured parts are missing
        prop = _as_text(row.get("Prop_Addr")).strip()
        if prop and prop.lower() != "nan":
            parts = [prop]

    return (" ".join(parts) + ", Ames, IA, USA") if parts else ""

# --- Guard: only try to build if we actually have rows to process
if len(geo_filtered) == 0:
    print("geo_filtered is empty — double-check your PID ↔ MapRefNo key normalization and overlap.")
else:
    geo_filtered = geo_filtered.copy()
    geo_filtered["FullAddress"] = geo_filtered.apply(build_full_address, axis=1)
    print(geo_filtered[["MapRefNo", "Prop_Addr", "PA-Nmbr", "PA-PreD", "PA-Strt", "PA-StSfx", "PA-PostD", "FullAddress"]].head(10))

Ames PID raw (col): ['909176150', '905476230', '911128020', '535377150', '534177230']
Geo MapRefNo raw (col): ['520400001', '520440010', '520440020', '520445001', '520445010']
Overlap count: 2558
Overlap sample: ['527146135', '906392120', '906475170', '904300150', '909253190', '534450090', '906382020', '528118090', '527357020', '916403230']
Filtered geo rows: 2602
      MapRefNo          Prop_Addr PA-Nmbr PA-PreD    PA-Strt PA-StSfx  \
113  526350040     3109 GROVE AVE  3109.0     NaN      GROVE      AVE   
118  526351010   630 RIVER OAK DR   630.0     NaN  RIVER OAK       DR   
120  526351030   620 RIVER OAK DR   620.0     NaN  RIVER OAK       DR   
127  526351100     3010 GROVE AVE  3010.0     NaN      GROVE      AVE   
272  526301100  3126 NORTHWOOD DR  3126.0     NaN  NORTHWOOD       DR   
276  526302030  3115 NORTHWOOD DR  3115.0     NaN  NORTHWOOD       DR   
277  526302040  3113 NORTHWOOD DR  3113.0     NaN  NORTHWOOD       DR   
284  526302110   623 RIVER OAK DR   623.0     NaN

In [14]:
# ============================================================
# Overnight Geocoding Script (Nominatim / OpenStreetMap)
# Requirements:
#   pip install geopy
#
# Input in memory:
  # ames_with_addr : pd.DataFrame with columns ["PID", "FullAddress"]
#
# Outputs on disk:
#   geocode_cache.csv     (running cache for unique addresses)
#   geocode_results.csv   (final unique address -> Lat, Lon)
#   ames_with_latlon.csv  (merge Lat/Lon back to your rows by FullAddress)
#   geocode_failures.csv  (addresses that failed geocoding)
# ============================================================

import os
import time
import math
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# -----------------------------
# 0) CONFIG — EDIT IF NEEDED
# -----------------------------
COL_PID       = "MapRefNo"          # your unique row identifier
COL_ADDR      = "FullAddress"  # full address string, e.g., "123 MAIN ST, Ames, IA, USA"

CACHE_PATH    = "geocode_cache.csv"     # resume file for (FullAddress, Lat, Lon)
RESULTS_PATH  = "geocode_results.csv"   # final unique address results
MERGED_PATH   = "ames_with_latlon.csv"  # merged back onto ames_with_addr
FAIL_PATH     = "geocode_failures.csv"  # addresses that never resolved

CHUNK_SIZE    = 400     # addresses per chunk (tune if you like)
FLUSH_EVERY   = 50      # write cache every N lookups for safety
MIN_DELAY_SEC = 1.2     # Nominatim policy ~1 req/sec (be polite)
MAX_RETRIES   = 3       # per-address retries

# A descriptive user_agent is required by Nominatim. Add a contact email if possible.
USER_AGENT    = "ames-housing-geocoder (contact: you@example.com)"

# -----------------------------
# 1) SANITY CHECKS
# -----------------------------

ames_with_addr = geo_filtered.copy()

if "ames_with_addr" not in globals():
    raise RuntimeError("Expected a DataFrame named `ames_with_addr` in memory.")

missing_cols = {COL_PID, COL_ADDR} - set(ames_with_addr.columns)
if missing_cols:
    raise RuntimeError(f"`ames_with_addr` is missing required columns: {missing_cols}")

if ames_with_addr.empty:
    raise RuntimeError("`ames_with_addr` is empty.")

# Drop obviously empty/NA addresses, and normalize to string
src = ames_with_addr.copy()
src[COL_ADDR] = src[COL_ADDR].astype(str).str.strip()
src = src[src[COL_ADDR].str.len() > 0].copy()

# -----------------------------
# 2) DEDUPE ADDRESSES
# -----------------------------
addr_unique = (
    src[[COL_ADDR]]
    .drop_duplicates()
    .reset_index(drop=True)
)

print(f"Unique addresses to geocode: {len(addr_unique):,}")

# -----------------------------
# 3) LOAD / PREP CACHE
# -----------------------------
if os.path.exists(CACHE_PATH):
    cache = pd.read_csv(CACHE_PATH)
    # Normalize cache schema and types
    cache = cache.rename(columns={
        COL_ADDR: COL_ADDR,
        "lat": "Lat",
        "lon": "Lon"
    })
    # Keep only needed cols; drop duplicates
    keep_cols = [c for c in [COL_ADDR, "Lat", "Lon"] if c in cache.columns]
    cache = cache[keep_cols].drop_duplicates(subset=[COL_ADDR], keep="last")
else:
    cache = pd.DataFrame(columns=[COL_ADDR, "Lat", "Lon"])

# Fast lookup set
cached_ok = set(cache[COL_ADDR].dropna().unique())
need_mask = ~addr_unique[COL_ADDR].isin(cached_ok)
to_geocode = addr_unique.loc[need_mask].reset_index(drop=True)

print(f"Already cached: {len(cached_ok):,} | Still need: {len(to_geocode):,}")

# -----------------------------
# 4) GEOCODER + RATE LIMITER
# -----------------------------
geolocator = Nominatim(user_agent=USER_AGENT, timeout=10)  # base timeout per request

# Wrap with RateLimiter for pacing + retries
geocode_raw = geolocator.geocode
geocode = RateLimiter(
    geocode_raw,
    min_delay_seconds=MIN_DELAY_SEC,   # spacing between calls
    max_retries=MAX_RETRIES,
    error_wait_seconds=5,              # wait between retry rounds
    swallow_exceptions=False           # raise so we can log failures
)

# -----------------------------
# 5) PROCESS IN CHUNKS
# -----------------------------

def geocode_one(addr: str):
    """Return dict with address, Lat, Lon (or NaNs) and error flag."""
    try:
        loc = geocode(addr, exactly_one=True, addressdetails=False)
        if loc is None:
            return {COL_ADDR: addr, "Lat": float("nan"), "Lon": float("nan"), "ok": False}
        return {COL_ADDR: addr, "Lat": loc.latitude, "Lon": loc.longitude, "ok": True}
    except Exception as e:
        # You can log e if desired
        return {COL_ADDR: addr, "Lat": float("nan"), "Lon": float("nan"), "ok": False}

# Work chunk-by-chunk
results_new = []  # only for this run (not the pre-existing cache)

if len(to_geocode) > 0:
    n_chunks = math.ceil(len(to_geocode) / CHUNK_SIZE)
    print(f"Processing in {n_chunks} chunk(s) of up to {CHUNK_SIZE} addresses…")

    processed = 0
    for ci in range(n_chunks):
        a = ci * CHUNK_SIZE
        b = min((ci + 1) * CHUNK_SIZE, len(to_geocode))
        chunk = to_geocode.iloc[a:b].reset_index(drop=True)

        print(f"[Chunk {ci+1}/{n_chunks}] {a}..{b-1} (size={len(chunk)})")
        t0 = time.time()

        for i, addr in enumerate(chunk[COL_ADDR].tolist(), start=1):
            rec = geocode_one(addr)
            results_new.append(rec)
            processed += 1

            # Periodically flush to disk (append to cache)
            if (len(results_new) % FLUSH_EVERY) == 0:
                tmp_df = pd.DataFrame(results_new)
                # Merge new into cache (dedupe by address; keep last)
                cache = pd.concat([cache, tmp_df[[COL_ADDR, "Lat", "Lon"]]], ignore_index=True)
                cache = cache.drop_duplicates(subset=[COL_ADDR], keep="last")
                cache.to_csv(CACHE_PATH, index=False)
                print(f"  Flushed {len(results_new)} new records | total cached={len(cache):,}")

        # End of chunk — flush what’s left
        if results_new:
            tmp_df = pd.DataFrame(results_new)
            cache = pd.concat([cache, tmp_df[[COL_ADDR, "Lat", "Lon"]]], ignore_index=True)
            cache = cache.drop_duplicates(subset=[COL_ADDR], keep="last")
            cache.to_csv(CACHE_PATH, index=False)
            print(f"  Chunk flushed | total cached={len(cache):,}")
            results_new = []

        t1 = time.time()
        print(f"  Chunk time: {t1 - t0:.1f}s | processed so far: {processed:,}")
else:
    print("Nothing new to geocode — using cache only.")

# -----------------------------
# 6) FINALIZE UNIQUE RESULTS
# -----------------------------
unique_results = cache.drop_duplicates(subset=[COL_ADDR], keep="last").reset_index(drop=True)
unique_results.to_csv(RESULTS_PATH, index=False)
print(f"Saved unique results -> {RESULTS_PATH} ({len(unique_results):,} rows)")

# Failures list
fail_df = unique_results[unique_results["Lat"].isna()].copy()
fail_df.to_csv(FAIL_PATH, index=False)
print(f"Failures saved -> {FAIL_PATH} ({len(fail_df):,} rows)")

# -----------------------------
# 7) MERGE BACK TO YOUR ROWS
# -----------------------------
merged = src.merge(unique_results, on=COL_ADDR, how="left")
merged.to_csv(MERGED_PATH, index=False)
print(f"Merged Lat/Lon -> {MERGED_PATH} ({len(merged):,} rows)")

# Quick success rate
succ_rate = merged["Lat"].notna().mean()
print(f"Geocoding success rate on all rows: {succ_rate:.1%}")

Unique addresses to geocode: 2,552
Already cached: 0 | Still need: 2,552
Processing in 7 chunk(s) of up to 400 addresses…
[Chunk 1/7] 0..399 (size=400)


/var/folders/jp/xgt314z12mgc2qh1wbn4mbx40000gn/T/ipykernel_14996/862216844.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cache = pd.concat([cache, tmp_df[[COL_ADDR, "Lat", "Lon"]]], ignore_index=True)


  Flushed 50 new records | total cached=50
  Flushed 100 new records | total cached=100
  Flushed 150 new records | total cached=150
  Flushed 200 new records | total cached=200
  Flushed 250 new records | total cached=250
  Flushed 300 new records | total cached=300
  Flushed 350 new records | total cached=350
  Flushed 400 new records | total cached=400
  Chunk flushed | total cached=400
  Chunk time: 497.0s | processed so far: 400
[Chunk 2/7] 400..799 (size=400)
  Flushed 50 new records | total cached=450
  Flushed 100 new records | total cached=500
  Flushed 150 new records | total cached=550
  Flushed 200 new records | total cached=600
  Flushed 250 new records | total cached=650
  Flushed 300 new records | total cached=700
  Flushed 350 new records | total cached=750
  Flushed 400 new records | total cached=800
  Chunk flushed | total cached=800
  Chunk time: 506.2s | processed so far: 800
[Chunk 3/7] 800..1199 (size=400)
  Flushed 50 new records | total cached=850
  Flushed 100 